In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
df_2014 = pd.read_csv("../data/raw/CDSarchive201403/Reference_CCAProfiles_2008_2012.csv")
df_2015 = pd.read_csv("../data/raw/CDSarchive201508/Reference_CCAProfiles_2009_2013.csv")
df_2016 = pd.read_csv("../data/raw/CDSarchive201606/Reference_CCAProfiles_2010_2014.csv")
df_2017 = pd.read_csv("../data/raw/CDSarchive201706/Reference_CCAProfiles_2011_2015.csv")
df_2018 = pd.read_csv("../data/raw/CDSarchive201811/Reference_CCAProfiles_2012_2016.csv")
df_2019 = pd.read_csv("../data/raw/CDSarchive201906/Reference_CCAProfiles_2013_2017.csv")
df_2020 = pd.read_csv("../data/raw/CDSarchive202007/ReferenceCCAProfiles20142018.csv")

In [16]:
# starting analysis with these columns
need_columns= ['GEOG', '2000_POP', '2010_POP', 'TOT_POP', 'YEAR', 'MEDINC', 'EMP', 'IN_LBFRC', 'OWN_OCC_HU', 
               'RENT_OCC_HU', 'VAC_HU', 'TOT_HH', 'HU_SNG_DET', 'HU_SNG_ATT', 'HU_2UN', 'HU_3_4UN', 'HU_GT_5UN', 
               'MED_HA', 'BR_0_1', 'BR_2', 'BR_3', 'BR_4', 'BR_5','WHITE', 'HISP', 'BLACK', 'ASIAN', 'OTHER']

In [458]:
#df_2020.drop(df_2020[df_2020['GEOG'].isna()].index, inplace=True)
#df_2020['HU_GT_5UN'] =  df_2020['HU_5_9UN'] + df_2020['HU_GT_19UN']

In [17]:
all_dfs ={2014: df_2014, 
          2015: df_2015, 
          2016: df_2016, 
          2017: df_2017,
          2018: df_2018,
          2019: df_2019,
          2020: df_2020}

years = [x for x in range(2014,2021)]

### Initial data cleaning

In [18]:
rename_columns = {"CCA": "GEOG",
                 'RNT_OCC_HU': 'RENT_OCC_HU',
                  'MED_INC' : 'MEDINC'
                 }
df_list = []
for y in years:
    df = all_dfs[y]
    ycol = [y]*len(df)
    df['YEAR'] = ycol
    df = df.rename(columns=rename_columns)
    
    c = df.columns.to_list()
    df = df.drop(columns=list(set(c) - set(need_columns)))

    df_list.append(df)
    df.to_csv("../data/clean/{}.csv".format(y))

In [19]:
for df in df_list:
    print(1)
    try:
        df[need_columns]
    except:
        print("missing")

1
1
1
1
1
1
1


In [20]:
df_all_data = pd.concat(df_list)
df_all_data.shape

(539, 28)

### Feature generation

##### predictor columns
- unemployment rate
- race
- median income
- income index relative to city overall?
- % rented housing units
- % owned housing units
- % or number vacant
- housing type
- housing age
- housing size


In [21]:
employment_rate = df_all_data['EMP']/df_all_data['IN_LBFRC']
df_all_data['EMP_RATE'] = employment_rate

In [22]:
race_columns = ['WHITE', 'HISP', 'BLACK', 'ASIAN', 'OTHER']

for race in race_columns:
    pct = df_all_data[race]/df_all_data['TOT_POP']
    name = race + "_PCT"
    df_all_data[name] = pct

In [23]:
rnt_own_columns = ['OWN_OCC_HU', 'RENT_OCC_HU', 'VAC_HU']

for r in rnt_own_columns:
    pct = df_all_data[r]/df_all_data['TOT_HH']
    name = r + "_PCT"
    df_all_data[name] = pct

In [24]:
df_all_data.to_csv("../data/clean/clean_data.csv")